In [1]:
from qdrant_client import QdrantClient
client = QdrantClient(path="/home/ubuntu/alphapolitica/db") 

In [ ]:
from qdrant_client.models import Distance, VectorParams
client.recreate_collection(
    collection_name="alphapolitica_face_db",
    vectors_config=VectorParams(size=128, distance=Distance.COSINE),
)

In [ ]:
import face_recognition
from PIL import Image, ImageDraw
import numpy as np
import glob
from tqdm import tqdm
from qdrant_client.models import PointStruct

failed = []
id_ = 0
for face_loc in tqdm(glob.glob('test_faces/*png')):
    try:
        image_ = face_recognition.load_image_file(face_loc)
        face_encoding = face_recognition.face_encodings(image_,  model= 'small')[0]
        client.upsert(
        collection_name="alphapolitica_face_db",
        points=[
            PointStruct(
                id=id_,
                vector=face_encoding.tolist(),
                payload={"location": face_loc}
            )
        ]
    )
    except:
        failed.append(face_loc) 
    id_ = id_ +1

In [ ]:
import face_recognition
from PIL import Image, ImageDraw
import numpy as np
import glob
from tqdm import tqdm
first_arr_loaded = True
for face_loc in ['m1.png']:#glob.glob('known_faces/*png'):
    print(face_loc)
    image_ = face_recognition.load_image_file(face_loc)
    face_locations = face_recognition.face_locations(image_)
    face_encoding = face_recognition.face_encodings(image_ , model= 'small')[0]
    if first_arr_loaded:
        known_faces_arr = np.reshape(face_encoding, (-1,len(face_encoding)))
        first_arr_loaded = False
    else:
        known_faces_arr = np.append(known_faces_arr,  np.reshape(face_encoding, (-1,len(face_encoding))), axis = 0) 

In [ ]:
query_vector = face_encoding
hits = client.search(
    collection_name="alphapolitica_face_db",
    query_vector=query_vector,
    score_threshold=0.9  # Return 5 closest points
)